In [1]:
import numpy as np
import pickle
# from Utils.TimeLogger import log
from scipy.sparse import csr_matrix
import time
import scipy.sparse as sp
minn = 1647180684
maxx = 0
import json
year_dict=dict()
def ok(year,month):
    global year_dict
    year_dict[year]=1
    if year <= 2001 and year>1996:
        return True
    else:
        return False
def transTime(date):
    timeArr = time.localtime(date)
    # timeArr = time.strptime(timeArray, '%Y-%m-%d %H:%M:%S')
    year = date# = time.mktime(timeArr)
    global minn
    global maxx
    # global minn_y
    # global maxx_y
    # minn_y = min(minn_y,timeArr.tm_year)
    # maxx_y = max(maxx_y,timeArr.tm_year)
    if (ok(timeArr.tm_year,timeArr.tm_mon)):
        minn = min(minn, year)
        maxx = max(maxx, year)        
        return year
    return None# time.mktime(timeArr)

In [3]:
from scipy.sparse import csr_matrix
intMat = csr_matrix(([0.1,0.1,0.1], ([0,0,0], [0,0,0])), shape=(2, 2))

In [87]:
print(year_dict)

{2001: 1, 2000: 1, 2002: 1, 2003: 1, 1996: 1, 1997: 1, 2006: 1, 2005: 1, 1999: 1, 1998: 1, 2007: 1, 2008: 1, 2004: 1, 2009: 1, 1995: 1}


In [3]:
def mapping(infile):
    
	usrId = dict()
	itmId = dict()
	usrid, itmid = [0, 0]
	interaction = list()
	with open(infile, 'r', encoding='utf-8') as fs:
		for line in fs:
			arr = line.strip().split('::')
			# print(arr)
			row = int(arr[0])
			col = int(arr[1])
			timeStamp = transTime(int(arr[-1]))
			if timeStamp is None:
				continue
			if row not in usrId:
				usrId[row] = usrid
				interaction.append(dict())
				usrid += 1
			if col not in itmId:
				itmId[col] = itmid
				itmid += 1
			usr = usrId[row]
			itm = itmId[col]
			if(itm not in interaction[usr]):
				interaction[usr][itm]=[]
			interaction[usr][itm].extend([timeStamp])
			# print(interaction)
	print(minn, maxx)
	return interaction, usrid, itmid

In [4]:
def checkFunc1(cnt):
	return cnt >= 5
def checkFunc2(cnt):
	return cnt >= 1
def checkFunc3(cnt):
	return cnt >= 1
def checkFunc4(cnt):
    return cnt <= 200
def filter(interaction, usrnum, itmnum, ucheckFunc, icheckFunc, filterItem=True):
    # get keep set
    usrKeep = set()
    itmKeep = set()
    itmCnt = [0] * itmnum
    for usr in range(usrnum):
        data = interaction[usr]
        usrCnt = 0
        for col in data:
            itmCnt[col] += 1
            usrCnt += 1
        if ucheckFunc(usrCnt):
            usrKeep.add(usr)
    for itm in range(itmnum):
        if not filterItem or icheckFunc(itmCnt[itm]):
            itmKeep.add(itm)

    # filter data
    retint = list()
    usrid = 0
    itmid = 0
    itmId = dict()
    user_id = list()
    item_id = list()
    time_int = list()
    for row in range(usrnum):
        if row not in usrKeep:
            continue
        usr = usrid
        usrid += 1
        retint.append(dict())
        data = interaction[row]
        for col in data:
            if col not in itmKeep:
                continue
            if col not in itmId:
                itmId[col] = itmid
                itmid += 1
            itm = itmId[col]
            retint[usr][itm] = data[col]
            for time_div in data[col]:
                user_id.append(usr+1)
                item_id.append(itm+1)
                time_int.append(int(time_div))
    return retint, usrid, itmid, user_id, item_id, time_int

In [12]:
def transtocsv(interaction, usrnum, itmnum):
    user_id = list()
    item_id = list()
    time_int = list()
    for row in range(usrnum):
        data = interaction[row]
        # print(data)
        for col in data:
            # print(col)
            if(data[col] is None):
                continue
            for time_div in data[col]:
                user_id.append(row+1)
                item_id.append(col+1)
                time_int.append(int(time_div))
    train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
    train.to_csv('train_movielens.csv',index=False,sep='\t')
    return 0#user_id,item_id,time_int
transtocsv(interaction, usrnum, itmnum)
# user_id, item_id, time_int = transtocsv(interaction, usrnum, itmnum)


0

In [57]:
train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
train.to_csv('train_movielens.csv',index=False,sep='\t')

In [2]:
def negSamp(temLabel, sampSize, nodeNum):
    negset = [None] * sampSize
    cur = 0
    while cur < sampSize:
        rdmItm = np.random.choice(nodeNum)
        if rdmItm not in temLabel:
            negset[cur] = rdmItm+1
            cur += 1
    return negset

def split(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]

    tstInt = [None] * usrnum
    trainPos = [None]* usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            if (itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print(usr)
            exception += 1
            continue
        # print('1',temp)
        # print(temp)
        temp.sort(key=lambda x: x[1])
        # print(temp)
        tstInt[usr] = temp[-1][0]
        # user_id.append(usr+1)
        # item_id.append(tstInt[usr]+1)
        # time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        # neg_items.append(negSamp(interaction[usr],99,itmnum))
        print(usr,tstInt[usr],interaction[usr][tstInt[usr]])
        if(len(interaction[usr][tstInt[usr]])==1):
            interaction[usr][tstInt[usr]] = None
        else:
            interaction[usr][tstInt[usr]] = interaction[usr][tstInt[usr]][1:]
    user_behavior_sequence=list()
    for usr in range(usrnum):
        temp = list()
        data = interaction[usr]
        for itm in data:
            if(itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print('2',usr)
            exception += 1
            continue
        #print(temp)
        temp.sort(key=lambda x: x[1])
        user_behavior_sequence.append(np.array(temp)[:-1,0])
        trainPos[usr] = temp[-1][0]
        print(usr,trainPos[usr],interaction[usr][trainPos[usr]])
        '''
        if(len(interaction[usr][trainPos[usr]])==1):
            interaction[usr][trainPos[usr]] = None
        else:
            interaction[usr][trainPos[usr]] = interaction[usr][trainPos[usr]][1:]
        '''
    # print(user_id,time_id,time_id,neg_items)
    # train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    # train.to_csv('test_yelp_4.csv',index=False,sep='\t')
    # print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt, trainPos, user_behavior_sequence

In [5]:
from pandas.core.frame import DataFrame
prefix = 'D:/edgedownload/'
print('Start')
minn = 1647180684
maxx = 0
minn_y = 2022
maxx_y = 2000
interaction, usrnum, itmnum = mapping('D:\\experiment\\movielens\\ratings.dat')
print('Id Mapped, usr %d, itm %d' % (usrnum, itmnum))


Start
852048853 1009814376
Id Mapped, usr 27382, itm 4896


In [ ]:
interaction0, usrnum0, itmnum0=interaction, usrnum, itmnum

In [ ]:
interaction, usrnum, itmnum=interaction0, usrnum0, itmnum0

In [6]:
checkFuncs = [checkFunc1, checkFunc2, checkFunc3, checkFunc4]
for i in range(len(checkFuncs)):
    filterItem = True if i < 1 else False
    interaction, usrnum, itmnum, user_id, item_id, time_int = filter(interaction, usrnum, itmnum, checkFuncs[i], checkFuncs[i], filterItem)
    print('Filter', i, 'times:', usrnum, itmnum)
    '''
    if(i==len(checkFuncs)):
        train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
        train.to_csv('train_amazon.csv',index=False,sep='\t')
        print(train)
    '''
        
print('Sparse Samples Filtered, usr %d, itm %d' % (usrnum, itmnum))
print(len(interaction))


Filter 0 times: 27088 4665
Filter 1 times: 27088 4665
Filter 2 times: 27088 4665
Filter 3 times: 22025 4619
Sparse Samples Filtered, usr 22025, itm 4619
22025


In [ ]:
train['eventdate']=pd.to_datetime(train['time'],unit='s')

In [58]:
def count_frequency(interaction,usernum,itmnum):
    count_item=np.zeros([itmnum],dtype=int)
    for i in range(itmnum):
        for j in range(usernum):
            if(i in interaction[j] and interaction[j][i] is not None):
                count_item[i]+=len(interaction[j][i])
    print(count_item)
    return np.argsort(count_item)

In [106]:
interaction[0][1]

In [107]:
ret=count_frequency(interaction,usrnum,itmnum)

[ 5324   738 10649 ...     1     1     1]


In [108]:
ret

array([4309, 4437, 4511, ...,   66,   16,    2], dtype=int64)

In [109]:
neg_frequency=ret

In [110]:
with open(prefix+'movie_sort', 'wb') as fs:
	pickle.dump(neg_frequency, fs)

In [10]:
with open(prefix+'movie_sort', 'rb') as fs:
	neg_frequency=pickle.load(fs)

In [ ]:
neg_frequency.max()

In [11]:
def negSamp_fre(temLabel, sampSize, nodeNum):
    negset = [None] * sampSize
    cur = 0
    i = 0
    while cur < sampSize:
        rdmItm = neg_frequency[-i]# np.random.choice(nodeNum)
        if rdmItm not in temLabel:
            negset[cur] = rdmItm+1
            cur += 1
        i += 1
    return negset

def split(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]

    tstInt = [None] * usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            temp.append((itm, data[itm]))
        if len(temp) == 0:
            exception += 1
            continue
        temp.sort(key=lambda x: x[1])
        tstInt[usr] = temp[-1][0]
        user_id.append(usr+1)
        item_id.append(tstInt[usr]+1)
        time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        neg_items.append(negSamp_fre(interaction[usr],99,itmnum))
        interaction[usr][tstInt[usr]] = None
        print(usr+1,tstInt[usr]+1,temp)
    # print(user_id,time_id,time_id,neg_items)
    train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    train.to_csv('test_amazon.csv',index=False,sep='\t')
    # print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt

In [ ]:
year_count={2004:0,2005:0,2006:0,2007:0,2008:0,2009:0,2010:0,2011:0,2012:0,2014:0,2015:0,2016:0,2017:0,2018:0,2019:0}
timelist=train['eventdate'].values
dates = pd.DatetimeIndex(train['eventdate'].values)
for i in range(len(timelist)):
    year_count[dates[i].year]+=1

In [ ]:
year_count

In [7]:
def trans_sub(interaction, usrnum, itmnum, gragh_num):
	global minn
	global maxx
	interval = (maxx-minn)/gragh_num
	rcd = [[list(), list(), list()]]
	for i in range(gragh_num-1):
		rcd.append([list(), list(), list()])
	timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)
	for usr in range(usrnum):
		if interaction[usr] == None:
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					gragh_no=int(((one_data-minn)/interval))
					if(gragh_no>=gragh_num):
						print(one_data,gragh_no)
						gragh_no=gragh_num-1
					# print(gragh_no,one_data)
					if(len(rcd[gragh_no][0])==0 or  rcd[gragh_no][0][-1]!=usr or rcd[gragh_no][1][-1]!=col):
						rcd[gragh_no][0].append(usr)
						rcd[gragh_no][1].append(col)
						rcd[gragh_no][2].append(one_data)
						# rcd[gragh_no][0].append(usrnum+col)
						# rcd[gragh_no][1].append(usr)
						# rcd[gragh_no][2].append(1.0)
						timeMat[usr,col]=gragh_no
	intMat=list()
	for i in range(gragh_num):
		intMat.append(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum, itmnum)))
		# intMat.append(normalized_adj_single(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum+itmnum, usrnum+itmnum))))+itmnum usrnum+
		print(intMat[i])
	return intMat, timeMat.tocsr()

In [8]:
# print(interaction, usrnum, itmnum), trainPos
trnInt, tstInt, trainPos, user_behavior_sequence = split(interaction, usrnum, itmnum)
print('Datasets Splited')
#print(trnInt)

4175 836 [949720084]
15503 16 [969727687]
14918 242 [864627984]
13137 568 [853795341]
12191 2667 [863272607]
4462 612 [986098494]
5320 985 [857030959]
17045 480 [860408752]
16339 299 [945416413]
10036 17 [852318277]
20727 2347 [980722887]
15506 119 [905554942]
4386 149 [961183619]
12672 80 [946624846]
10899 726 [938880189]
20418 1505 [967032639]
144 1213 [945886544]
5346 756 [943180761]
21876 846 [941921084]
15351 563 [956117441]
2483 16 [858764493]
8833 297 [944582459]
9784 1463 [975009938]
9383 2362 [889468582]
1006 58 [868818571]
2173 701 [961825851]
19266 397 [974903558]
4686 297 [995099999]
2002 16 [858593446]
19405 1735 [940368472]
5323 3197 [1005567905]
10962 966 [862685052]
16012 645 [865125530]
21708 66 [948053574]
6015 2795 [1005607019]
18840 94 [856393941]
4457 354 [986098101]
1401 182 [869024142]
8253 957 [991837144]
4205 979 [879497487]
8 379 [857927283]
14774 1771 [994782465]
8151 568 [861285465]
13899 230 [994301394]
18508 774 [856213259]
14852 296 [967912127]
19338 728 

C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_42992/554800582.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  user_behavior_sequence.append(np.array(temp)[:-1,0])


 [948579844]
574 2430 [1002006878]
575 702 [941889697]
576 535 [862916053]
577 831 [948582219]
578 383 [945955852]
579 636 [913231144]
580 94 [948577720]
581 115 [981919080]
582 1690 [858099875]
583 452 [974423242]
584 409 [1001489647]
585 48 [858100090]
586 498 [913232677]
587 906 [950157122]
588 1849 [858100725]
589 1843 [942323305]
590 23 [948588833]
591 355 [868524118]
592 2669 [942165586]
593 565 [858102980]
594 72 [913233035]
595 249 [974423824]
596 2171 [874360938]
597 230 [945992176]
598 230 [981930181]
599 879 [853929519]
600 4 [960347624]
601 2078 [981930869]
602 58 [948587641]
603 114 [868528629]
604 2316 [941730216]
605 230 [945961758]
606 354 [858103458]
607 1493 [950220205]
608 910 [869606503]
609 749 [941733725]
610 235 [960355691]
611 186 [913246153]
612 90 [948593594]
613 468 [868533486]
614 1415 [960358056]
615 837 [913251228]
616 1962 [984324918]
617 261 [941761168]
618 332 [945963121]
619 299 [960699331]
620 1771 [948596779]
621 1226 [1005530135]
622 560 [867346960]

In [ ]:
print(len(trnInt))

In [ ]:
print(trainPos)

In [ ]:
print(user_behavior_sequence[0])

In [ ]:
tstInt

In [104]:
with open(prefix+'trn_pos_movie', 'wb') as fs:
	pickle.dump(trainPos, fs)

In [102]:
with open(prefix+'tstInt_movie', 'wb') as fs:
	pickle.dump(tstInt, fs)

In [103]:
with open(prefix+'tstInt4_amazon', 'rb') as fs:
	tst_Int=pickle.load(fs)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/edgedownload/tstInt4_amazon'

In [ ]:
tstInt

In [9]:
minn = 1647180684
maxx = 0

def trans(interaction, usrnum, itmnum):
	global maxx,minn
	count=0
	r, c, d = [list(), list(), list()]
	for usr in range(usrnum):
		if interaction[usr] == None:
			count+=1
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					if(one_data>maxx):
						maxx=one_data
					if(one_data<minn):
						minn=one_data
					r.append(usr)
					c.append(col)
					d.append(1.0)
				else:
					count+=1
	print(max(r),max(c))
	print(count)
	intMat = csr_matrix((d, (r, c)), shape=(usrnum, itmnum))
	return intMat

In [10]:
print(usrnum,itmnum)
trnMat = [trans(trnInt, usrnum, itmnum)]
print(maxx,minn)

subMat,timeMat=trans_sub(trnInt, usrnum, itmnum, 12)
print(timeMat)
trnMat.append(subMat)
trnMat.append(timeMat)
print('Train Mat Done')
with open(prefix+'trn_mat_time_movie12', 'wb') as fs:
	pickle.dump(trnMat, fs)

22025 4619
22024 4618
1506549
1009814161 852048853


C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_42992/2970320320.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)


1009814161 12
1009814161 12
  (1, 6)	857911264
  (1, 9)	857911264
  (1, 10)	857911264
  (1, 13)	857912738
  (1, 15)	857911357
  (1, 18)	857911690
  (1, 20)	857911264
  (1, 21)	857911357
  (1, 22)	857911265
  (1, 23)	857913507
  (1, 24)	857911752
  (1, 25)	857911264
  (1, 26)	857912492
  (1, 27)	857911416
  (1, 28)	857912840
  (1, 29)	857911416
  (1, 30)	857912791
  (1, 31)	857912365
  (1, 32)	857911264
  (1, 33)	857912492
  (1, 34)	857912933
  (1, 35)	857912441
  (1, 36)	857912933
  (1, 37)	857912535
  (1, 38)	857912738
  :	:
  (22006, 481)	857905800
  (22006, 642)	857905874
  (22006, 644)	857905907
  (22006, 766)	857905855
  (22010, 2)	857905969
  (22010, 6)	857905916
  (22010, 10)	857905916
  (22010, 15)	857905944
  (22010, 22)	857905916
  (22010, 25)	857905916
  (22010, 27)	857905970
  (22010, 32)	857905916
  (22010, 68)	857906006
  (22010, 144)	857905944
  (22010, 349)	857906061
  (22010, 354)	857905970
  (22010, 357)	857906162
  (22010, 377)	857906085
  (22010, 641)	857906060
  (2

In [11]:
trnMat[1][:]

[<22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 240381 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 83690 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 45398 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 52000 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 29337 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 17450 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 186970 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of type '<class 'numpy.intc'>'
 	with 271252 stored elements in Compressed Sparse Row format>,
 <22025x4619 sparse matrix of

In [ ]:
yelp_sequence=list()
for i in range(len(user_behavior_sequence)):
    indexs=np.array(range(len(user_behavior_sequence[i])))
    indexs=np.expand_dims(indexs,axis=1)
    
    yelp_sequence.append(np.concatenate((indexs,np.expand_dims(user_behavior_sequence[i],axis=1)),axis=1))

In [ ]:
np.random.choice(yelp_sequence[0],5)

In [ ]:
with open(prefix+'yelp_sequence', 'wb') as fs:
	pickle.dump(user_behavior_sequence, fs)

In [ ]:
print(trnMat[0].shape)

In [ ]:
print(interaction[0])

In [ ]:
import pandas as pd
train=pd.read_csv('train_amazon.csv',sep='\t')
train
test=pd.read_csv('test_amazon.csv',sep='\t')
test

In [20]:
tstInt

[None,
 100,
 123,
 225,
 None,
 None,
 None,
 357,
 379,
 None,
 None,
 None,
 549,
 587,
 None,
 573,
 None,
 None,
 None,
 None,
 None,
 846,
 32,
 851,
 None,
 None,
 None,
 None,
 None,
 912,
 964,
 980,
 1048,
 1064,
 1081,
 None,
 None,
 None,
 1100,
 None,
 None,
 None,
 742,
 None,
 None,
 None,
 128,
 None,
 None,
 1294,
 691,
 4,
 292,
 712,
 1085,
 None,
 1339,
 151,
 None,
 511,
 None,
 None,
 None,
 None,
 None,
 1330,
 None,
 None,
 1455,
 None,
 None,
 1497,
 None,
 583,
 292,
 690,
 230,
 None,
 305,
 None,
 None,
 155,
 706,
 None,
 None,
 453,
 890,
 None,
 444,
 None,
 None,
 1282,
 1648,
 None,
 None,
 None,
 1693,
 1420,
 None,
 None,
 None,
 233,
 1713,
 513,
 None,
 None,
 None,
 1259,
 1756,
 None,
 None,
 None,
 None,
 None,
 None,
 471,
 None,
 182,
 123,
 None,
 None,
 None,
 None,
 None,
 1755,
 818,
 None,
 438,
 513,
 945,
 None,
 1377,
 None,
 1972,
 None,
 None,
 None,
 None,
 None,
 1998,
 1004,
 None,
 446,
 None,
 1213,
 1092,
 904,
 2058,
 None,
 10

In [ ]:
maxx=train.time.max()
minn=train.time.min()

In [ ]:
df_clear = train.drop(train[(train['user_id']==test.user_id[0]) &(train['item_id']==test.item_id[0]) ].index)

In [ ]:
df_clear[df_clear.user_id==test.user_id[0]]


In [ ]:
train[train.user_id==test.user_id[0]]

In [ ]:
len(df_clear)

In [ ]:
len(train)

In [ ]:
len(train[(train['user_id']==test.user_id[0]) &(train['item_id']==test.item_id[0]) ])

In [ ]:
for i in range(1,10000):
    # print(test.item_id[i],test.user_id[i])
    df_clear.drop(df_clear[(df_clear['user_id']==test.user_id[i]) &(df_clear['item_id']==test.item_id[i]) ].index,inplace=True)

In [ ]:
import pandas as pd
test=pd.read_csv('D:/edgedownload/yelp_review',sep='\t')
test

In [ ]:
df_clear[df_clear.user_id==4866]

In [ ]:
train[train.user_id==4866]

In [ ]:
df_clear.to_csv('train_gowalla_srgnn.csv',index=False,sep='\t')

In [ ]:
import pickle
train_data = pickle.load(open('D:/experiment/SR-GNN-master/datasets/yoo/train.txt', 'rb'))

In [ ]:
maxxx=0
for i in range(len(train_data[0])):
    # print(len(train_data[0][i]))
    maxxx=max(maxxx,len(train_data[0][i]))
maxxx

In [ ]:
len(train_data[0])

In [ ]:
train_data2 = pickle.load(open('D:/edgedownload/yelp_review', 'rb'))

In [ ]:
maxxx=0
for i in range(len(train_data2[0])):
    # print(len(train_data[0][i]))
    maxxx=max(maxxx,len(train_data2[0][i]))
maxxx

In [ ]:
len(train_data2[0])

In [ ]:
import pandas as pd
train=pd.read_csv('D:\experiment\ReChorus-master\data\yelp_lyx\\train.csv',sep='\t')
train

In [ ]:
train['rank']=0

In [ ]:
order = ['user_id','item_id','rank','time']
train = train[order]

In [ ]:
train.to_csv('D:\experiment\BPR-pytorch-master\\yelp_train.csv',header=False,index=False,sep='\t')

In [ ]:
import pandas as pd
test=pd.read_csv('D:\experiment\ReChorus-master\data\yelp_lyx\\test.csv',sep='\t')
test['rank']=1
order = ['user_id','item_id','rank','time']
test = test[order]

In [ ]:
test.to_csv('D:\experiment\BPR-pytorch-master\\yelp_test.csv',header=False,index=False,sep='\t')

In [ ]:
test

In [ ]:
test=pd.read_csv('D:\experiment\ReChorus-master\data\yelp_lyx\\test.csv',sep='\t')
test

In [ ]:
test_neg=pd.DataFrame()
test_neg['1']=(test['user_id'],test['item_id'])

In [ ]:
test_neg=pd.DataFrame()
test_neg['1']=list(zip(test['user_id'].values, test['item_id'].values))


In [ ]:
test_neg

In [ ]:
test_neg['2']=test.neg_items

In [ ]:
test_neg

In [ ]:
test_neg.to_csv('D:\experiment\BPR-pytorch-master\\yelp_test_neg.csv',header=False,index=False,sep='\t')

In [ ]:
negs=np.concatenate(neg_list,axis=1)

In [ ]:
import ast
neg_list=[]
for i in range(10000):
    neg_list.append(ast.literal_eval(test.neg_items.values[i]))

In [ ]:
neg_list

In [ ]:
neg_list2=[]
for i in range(99):
    neg_list2.append(neg_list[:,i])

In [ ]:
neg_list=np.array(neg_list)

In [ ]:
neg_list2[1].shape

In [ ]:
for i in range(99):
    test_neg['a'+str(i)]=neg_list2[i]


In [ ]:
test_neg.drop(columns=['2'],inplace=True) 

In [ ]:
test_neg.to_csv('D:\experiment\BPR-pytorch-master\\yelp_test_neg.csv',header=False,index=False,sep='\t')

In [ ]:
sort1 = pickle.load(open('D:/edgedownload/amazon_sortnew', 'rb'))

In [ ]:
sort1

In [ ]:
sort2 = pickle.load(open('D:/edgedownload/amazon_sort', 'rb'))

In [ ]:
sort2